In [3]:
import sys
sys.path.append("/home/xjtu/.research/tools/")

from pdx.infer import DETECT

In [4]:
DETECT(model_path="/home/xjtu/.research/models/zybds_general_20210830", \
                 use_gpu=True, \
                 gpu_mem=100, \
                 gpu_id=0, \
                 confThre=0.8, \
                 label_dict={1:"cat", 2:"dog"},\
                 maxqsize=0)

--- Running analysis [ir_graph_build_pass]
--- Running analysis [ir_graph_clean_pass]
--- Running analysis [ir_analysis_pass]
--- Running IR pass [is_test_pass]
--- Running IR pass [simplify_with_basic_ops_pass]
--- Running IR pass [conv_affine_channel_fuse_pass]
--- Running IR pass [conv_eltwiseadd_affine_channel_fuse_pass]
--- Running IR pass [conv_bn_fuse_pass]
I0905 19:29:51.905046 2069490 graph_pattern_detector.cc:91] ---  detected 101 subgraphs
--- Running IR pass [conv_eltwiseadd_bn_fuse_pass]
--- Running IR pass [embedding_eltwise_layernorm_fuse_pass]
--- Running IR pass [multihead_matmul_fuse_pass_v2]
--- Running IR pass [squeeze2_matmul_fuse_pass]
--- Running IR pass [reshape2_matmul_fuse_pass]
--- Running IR pass [flatten2_matmul_fuse_pass]
--- Running IR pass [map_matmul_to_mul_pass]
--- Running IR pass [fc_fuse_pass]
--- Running IR pass [fc_elementwise_layernorm_fuse_pass]
--- Running IR pass [conv_elementwise_add_act_fuse_pass]
I0905 19:29:52.000846 2069490 graph_pattern_

In [2]:
def pinfer_preprocess(image, img_size):
    
    def resize(img, target_size):
        """resize to target size"""
        if not isinstance(img, np.ndarray):
            raise TypeError('image type is not numpy.')
        im_shape = img.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        im_scale_x = float(target_size) / float(im_shape[1])
        im_scale_y = float(target_size) / float(im_shape[0])
        img = cv2.resize(img, None, None, fx=im_scale_x, fy=im_scale_y)
        return img


    def normalize(img, mean, std):
        img = img / 255.0
        mean = np.array(mean)[np.newaxis, np.newaxis, :]
        std = np.array(std)[np.newaxis, np.newaxis, :]
        img -= mean
        img /= std
        return img

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    img = resize(image, img_size)
    img = img[:, :, ::-1].astype('float32')  # bgr -> rgb
    img = normalize(img, mean, std)
    img = img.transpose((2, 0, 1))  # hwc -> chw
    
    scale_factor = np.array([img_size*1. / image.shape[0], \
                         img_size*1. / image.shape[1]]).reshape((1, 2)).astype(np.float32)
    
    img_shape = np.array([img_size, img_size]).reshape((1, 2)).astype(np.float32)
    
    return [img_shape, img[np.newaxis, :], scale_factor]

In [3]:
def pinfer_init_predictor(use_gpu=True):
    model_dir = "/home/xjtu/.research/models/zybds_general_20210830/"
    model_file = os.path.join(model_dir, "model.pdmodel")
    params_file = os.path.join(model_dir, "model.pdiparams")

    config = Config(model_file, params_file)
    config.enable_memory_optim()
    if use_gpu:
        config.enable_use_gpu(100, 0)
    else:
        # If not specific mkldnn, you can set the blas thread.
        # The thread num should not be greater than the number of cores in the CPU.
        config.set_cpu_math_library_num_threads(4)
        config.enable_mkldnn()

    predictor = create_predictor(config)
    return predictor


def pinfer_run(predictor, img):
    # copy img data to input tensor
    input_names = predictor.get_input_names()
    for i, name in enumerate(input_names):
        input_tensor = predictor.get_input_handle(name)
        input_tensor.reshape(img[i].shape)
        input_tensor.copy_from_cpu(img[i].copy())

    # do the inference
    predictor.run()

    results = []
    # get out data from output tensor
    output_names = predictor.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = predictor.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)
    return results

In [4]:
img_size = 608

test_image = "/home/xjtu/.research/test.png"
image = cv2.imread(test_image)
[img_shape, image, scale_factor] = preprocess(image, img_size)

pred = init_predictor()

W0905 16:53:53.310479 1853814 analysis_predictor.cc:715] The one-time configuration of analysis predictor failed, which may be due to native predictor called first and its configurations taken effect.
--- Running analysis [ir_graph_build_pass]
--- Running analysis [ir_graph_clean_pass]
--- Running analysis [ir_analysis_pass]
--- Running IR pass [is_test_pass]
--- Running IR pass [simplify_with_basic_ops_pass]
--- Running IR pass [conv_affine_channel_fuse_pass]
--- Running IR pass [conv_eltwiseadd_affine_channel_fuse_pass]
--- Running IR pass [conv_bn_fuse_pass]
I0905 16:53:53.606518 1853814 graph_pattern_detector.cc:91] ---  detected 101 subgraphs
--- Running IR pass [conv_eltwiseadd_bn_fuse_pass]
--- Running IR pass [embedding_eltwise_layernorm_fuse_pass]
--- Running IR pass [multihead_matmul_fuse_pass_v2]
--- Running IR pass [squeeze2_matmul_fuse_pass]
--- Running IR pass [reshape2_matmul_fuse_pass]
--- Running IR pass [flatten2_matmul_fuse_pass]
--- Running IR pass [map_matmul_to_mu

In [6]:
%time
result = run(pred, [img_shape, image, scale_factor])
print(result)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
[array([[1.00000000e+00, 3.12242836e-01, 3.27871521e+02, 7.65298645e+02,
        4.14761475e+02, 8.70574280e+02],
       [0.00000000e+00, 2.82918960e-01, 2.48309723e+02, 7.47864319e+02,
        4.42070343e+02, 1.13743640e+03],
       [1.00000000e+00, 2.18931898e-01, 1.02238885e+03, 6.53739136e+02,
        1.09880005e+03, 7.46900024e+02],
       [0.00000000e+00, 1.44906744e-01, 1.04162354e+03, 5.38035645e+02,
        1.23196045e+03, 9.46592041e+02],
       [0.00000000e+00, 1.13129392e-01, 7.22261902e+02, 5.35396179e+02,
        8.85805237e+02, 8.76661926e+02],
       [1.00000000e+00, 1.06762908e-01, 6.59490723e+02, 6.41068481e+02,
        7.29434937e+02, 7.23398071e+02],
       [1.00000000e+00, 9.47438776e-02, 7.95251404e+02, 5.47262451e+02,
        8.66060486e+02, 6.28998535e+02],
       [0.00000000e+00, 7.84797370e-02, 8.61504089e+02, 6.51384094e+02,
        1.10976794e+03, 1.07188770e+03],
       [0.00000000e+00, 7.0